In [ ]:
import BBHX_PhenomD
import time
import numpy as np
from pycbc.waveform import get_td_waveform, get_fd_waveform
from pycbc.catalog import Merger
from pycbc.filter import resample_to_delta_t, highpass, matched_filter, sigma
import pycbc.psd
import matplotlib.pyplot as plt
from astropy.cosmology import Planck13 as cosmo
import pycbc.waveform.spa_tmplt as pyspt
from pycbc import waveform
from pycbc.inference.io import loadfile
import pandas
import h5py
import pickle
from bbhx.utils.constants import *
from bbhx.utils.transform import *
from pycbc.conversions import  mchirp_from_mass1_mass2
import astropy.coordinates

In [ ]:
## Load the results files from the inference, you may need to cd into where the hrd files were saved ##

##%cd FOLDER PATH, for example /mnt/d/INFERENCE_FOLDER/


def lf(file):           
    return loadfile("{}".format(file),"r")
results_A ={}
for i in range(10):
    results_A[i] = lf("lisa_smbhb_GAL_A_{}.hdf".format(i))
results_A

results_B ={}
for i in range(10):
    results_B[i] = lf("lisa_smbhb_GAL_B_{}.hdf".format(i))
    
    
    
def bayes_results(filename):
    bayes_factor = np.array([results_A[i].log_evidence[0] - results_B[i].log_evidence[0] for i in range(10)])
    bayes_factor_error = [np.sqrt(results_A[i].log_evidence[1]**2 + results_B[i].log_evidence[1]**2) for i in range(10)]
    np.savez(f"{filename}.npz",bf = bayes_factor, ang = ang_sep, err=bayes_factor_error )
    return bayes_factor,bayes_factor_error
  ## This function returns our LOG(BF) and its error (added in quadrature), and saves it if we want to compare results ##

In [ ]:
with open('false_dict.pkl', 'rb') as f:
    GAL_B = pickle.load(f)    
GAL_B

with open('merge_dict_A.pkl', 'rb') as f:
    GAL_A = pickle.load(f)
## Unpickle the dictionaries we made in "waveform.py" ##
    
    


dict_of_df_B = {}
df_keys = ['distance',
 'inclination',
 'loglikelihood',
 'logwt',
 'mchirp',
 'spin1z',
 'spin2z']

### Here we translate our dictionaries into pandas dataframes for ease of use ###
### GAL_B contains the TRUE parameters, dict_of_df contains the inference results ###

for i,merger in enumerate(results_B):
    dict_of_df_B[merger] = pandas.DataFrame(np.array([results_B[merger]["samples"][i] for i in list(results_B[i]['samples'].keys())])).T
    dict_of_df_B[merger].columns =['distance',
 'inclination',
 'loglikelihood',
 'logwt',
 'mchirp',
 'spin1z',
 'spin2z']

In [ ]:
true_dist = cosmo.luminosity_distance(0.5).value
ang_sep = [astropy.coordinates.angular_separation(true_long,true_lat,gal_b_long[i],gal_b_lat[i]) for i in range(10)]

true_lat = GAL_B[0]["beta"]
true_long  = GAL_B[0]["lam"]
true_inc = 1.GAL_B[0]["inc"]

gal_b_long = np.array([GAL_B[i]["lam"] for i in range(10)]) #### These are our TRUE PARAMETERS ###
gal_b_lat  = np.array([GAL_B[i]["beta"] for i in range(10)])
gal_b_spin1_true = np.array([GAL_B[i]["chi1z"] for i in range(10)])
gal_b_spin2_true = np.array([GAL_B[i]["chi2z"] for i in range(10)])
gal_b_dist_true = np.array([GAL_B[i]["dist"] / (PC_SI * 1e6) for i in range(10)])
gal_b_true_mass = np.array([mchirp_from_mass1_mass2(GAL_B[i]["m1"],GAL_B[i]["m2"]) for i in range(10)])




gal_b_spin1z =  np.array([dict_of_df_B[i]["spin1z"].iloc[-1] for i in range(10)])  
gal_b_spin2z =  np.array([dict_of_df_B[i]["spin2z"].iloc[-1] for i in range(10)])  
gal_b_inc =  np.array([dict_of_df_B[i]["inclination"].iloc[-1] for i in range(10)])  
gal_b_mass =  np.array([dict_of_df_B[i]["mchirp"].iloc[-1] for i in range(10)])
gal_b_dist =  np.array([dict_of_df_B[i]["distance"].iloc[-1] for i in range(10)]) 

In [ ]:
def bayes_results(filename):
    bayes_factor = np.array([results_A[i].log_evidence[0] - results_B[i].log_evidence[0] for i in range(10)])
    bayes_factor_error = [np.sqrt(results_A[i].log_evidence[1]**2 + results_B[i].log_evidence[1]**2) for i in range(10)]
    np.savez(f"{filename}.npz",bf = bayes_factor, ang = ang_sep, err=bayes_factor_error )
    return bayes_factor,bayes_factor_error

    

In [ ]:
def save_res(filename):
    np.savez(f"{filename}.npz",bf = bayes_factor, ang = ang_sep, err=bayes_factor_error )

In [ ]:
def prob_statement(plot=False):
    bayes_factor.sort()
    ang_sep.sort()
    plt.figure(dpi = 200,figsize = (10,8))
    points_within_centre_circle = []
    point_within_torus = []
    for idx,i in enumerate(ang_sep):
        point_counter = 0
        point_counter_circle = 0
        angle = np.linspace( 0 , 2*np.pi , 100 ) 

        radius = i
        small_r = ang_sep[idx]
        large_r = ang_sep[idx-1]
        for j in range(len(data_x)):
            if (data_x[j] - np.pi/2)**2 + (data_y[j] - 0 )**2 > large_r **2 and (data_x[j] - np.pi/2)**2 + (data_y[j] -  0)**2 < small_r **2:
                point_counter+=1
        point_within_torus.append(point_counter)
        #print(point_counter)

        
        x = radius * np.cos( angle ) +np.pi/2
        y = radius * np.sin( angle ) 
        if bayes_factor[idx]>=0 and plot ==True:
            plt.plot(x,y,'g',alpha=1,label =  f"{bayes_factor[idx]:.2f}")
        elif bayes_factor[idx]<=0 and plot == True:
            plt.plot(x,y,'r--',alpha=1,label =  f"{bayes_factor[idx]:.2f}")
        for j in range(len(data_x)):
            if (data_x[j] - np.pi/2)**2 + (data_y[j] -  0 )**2 < radius**2 and idx == 0 :
                point_counter_circle += 1
        points_within_centre_circle.append(point_counter_circle)
        #print(point_counter_circle)
        #plt.plot(x,y)

    legend=plt.legend(title="Ln(BF) for each ring",fontsize = 15)
    plt.setp(legend.get_title(),fontsize='x-large')
    #plt.xlim(np.pi/2)
    #plt.ylim(np.pi/2)


    point_within_torus[0] = points_within_centre_circle[0]
    point_within_torus = np.array(point_within_torus)
    points_unresolvable = sum(point_within_torus[bayes_factor<=1])
    print("Fraction unresolvable:",points_unresolvable/500 * 100



    if plot == True:
        plt.scatter(data_x,data_y,s=10.5)
        plt.scatter(np.pi/2,0,s=80,marker='+')
        plt.xlabel("Elliptic longitude (radians)",fontsize = 24)
        plt.ylabel("Elliptic latitude(radians)",fontsize = 24)
        plt.xticks(fontsize = 18)
        plt.yticks(fontsize = 18)
          
 inv_bayes_factor = np.array([results_B[i].log_evidence[0] - results_A[i].log_evidence[0] for i in range(10)])
    p_b = np.exp(sorted(inv_bayes_factor,reverse=True)) / point_within_torus
    
          
          
    if plot == True:
        plt.figure(dpi= 250,figsize = (12,10))
        plt.plot(ang_sep,p_b)
        plt.xticks(np.linspace(min(ang_sep),max(ang_sep),5),fontsize =18)
        plt.yticks(fontsize = 18)
        plt.xlabel("Angular separation (radians)",fontsize = 20)
        plt.ylabel("Relative probability of galaxy B containing the event",fontsize = 20)
        plt.grid()

    